<a href="https://colab.research.google.com/github/ACM-Research/interactive-game-development/blob/main/hand_game_controls.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Follow the steps in the order they are specified!**
Tutorial followed: https://medium.com/analytics-vidhya/training-a-model-for-custom-object-detection-tf-2-x-on-google-colab-4507f2cc6b80

In [ ]:
#Step 1: Mount ACM Hands Team Google Drive for Colab use.
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
#Step 2: For symbolically linking "/mydrive" call to ACM Hands Team Google Drive.
!ln -s /content/drive/MyDrive/ /mydrive
!ls /mydrive

'Colab Notebooks'   result.avi
 customTF2	   'Screen Recording 2022-11-04 at 8.53.01 PM.mp4'
 IMG_6272.JPG


In [ ]:
#Step 3:
#Clone TensorFlow model garden fork from ProjectSkyapple
!git clone --q https://github.com/ProjectSkyapple/models.git

# navigate to /models/research folder to compile protos
%cd /content/models/research

# Compile protos.
# !protoc object_detection/protos/*.proto --proto_path=object_detection/protos --python_out=object_detection/protos

# Install TensorFlow Object Detection API.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

/content/models/research
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 13.2 MB 8.3 MB/s 
     |████████████████████████████████| 352 kB 73.7 MB/s 
     |████████████████████████████████| 2.2 MB 51.7 MB/s 
     |████████████████████████████████| 25.0 MB 1.4 MB/s 
     |████████████████████████████████| 67 kB 6.5 MB/s 
     |████████████████████████████████| 116 kB 56.8 MB/s 
     |████████████████████████████████| 1.3 MB 59.1 MB/s 
     |████████████████████████████████| 

In [ ]:
#Step 4: Import modules.
import os
import glob
import xml.etree.ElementTree as ET
import pandas as pd
import tensorflow as tf

In [ ]:
#If needed: Add missing builder.py to Protobuf package
%pip install --upgrade protobuf
%cd "/usr/local/lib/python3.7/dist-packages/google/protobuf/internal"
!cp ./builder.py /usr/local/lib/python3.7/dist-packages/builder.py
%pip install --upgrade protobuf==3.19.6

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 408 kB 5.1 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.10.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.21.9 which is incompatible.
tensorflow-metadata 1.10.0 requires protobuf<4,>=3.13, but you have protobuf 4.21.9 which is incompatible.
tensorboard 2.10.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.21.9 which is incompatible.
apache-beam 2.43.0 requires protobuf<4,>3.12.2, but you have protobuf 4.21.9 which is incompatible.
/usr/local/lib/python3.7/dist-packages/google/protobuf/internal
Looking in indexes: https://py

In [ ]:
!mv /usr/local/lib/python3.7/dist-packages/builder.py /usr/local/lib/python3.7/dist-packages/google/protobuf/internal

In [ ]:
%cd /content/models/research
!python object_detection/builders/model_builder_tf2_test.py

In [ ]:
#Step 5a: Reload TensorBoard for monitoring training.
%reload_ext tensorboard
%tensorboard --logdir '/mydrive/customTF2/training_v4'

In [ ]:
#Step 5b: Train the dataset.
%cd /content/models/research/object_detection
!python model_main_tf2.py --pipeline_config_path=/mydrive/customTF2/data/pipeline_v4.config --model_dir=/mydrive/customTF2/training_v4 --alsologtostderr

In [ ]:
%cd /content/models/research/object_detection

/content/models/research/object_detection


In [ ]:
# Export inference graph.
%cd /content/models/research/object_detection
!python exporter_main_v2.py --trained_checkpoint_dir=/mydrive/customTF2/training_v3 --pipeline_config_path=/mydrive/customTF2/data/pipeline_v3.config --output_directory /mydrive/customTF2/data/inference_graph_v3

In [ ]:
import os
import time
import tensorflow as tf
import cv2
import numpy as np

from utils import label_map_util
from PIL import Image
from google.colab.patches import cv2_imshow
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

from IPython.display import HTML
from base64 import b64encode

#Path to saved model  

PATH_TO_SAVED_MODEL = "/mydrive/customTF2/data/inference_graph_v1/saved_model"

# Load label map and obtain class names and ids
#label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
category_index=label_map_util.create_category_index_from_labelmap("/mydrive/customTF2/data/label_map.pbtxt",use_display_name=True)


def visualise_on_image(image, bboxes, labels, scores, thresh):
    (h, w, d) = image.shape
    for bbox, label, score in zip(bboxes, labels, scores):
        if score > thresh:
            xmin, ymin = int(bbox[1]*w), int(bbox[0]*h)
            xmax, ymax = int(bbox[3]*w), int(bbox[2]*h)

            cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (0,255,0), 2)
            cv2.putText(image, f"{label}: {int(score*100)} %", (xmin, ymin), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
    return image

if __name__ == '__main__':
    
    # Load the model
    print("Loading saved model ...")
    detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)
    print("Model Loaded!")
    
    # Video Capture (video_file)
    video_capture = cv2.VideoCapture("/mydrive/customTF2/data/images/IMG_6277.mp4")
    start_time = time.time()
    
    frame_width = int(video_capture.get(3))
    frame_height = int(video_capture.get(4))
    #fps = int(video_capture.get(5))
    size = (frame_width, frame_height)
    
    #Initialize video writer
    result = cv2.VideoWriter('/mydrive/customTF2/nov_7_test.avi', cv2.VideoWriter_fourcc(*'MJPG'),15, size)

    while True:
      ret, frame = video_capture.read()
      if not ret:
          print('Unable to read video / Video ended')
          break
    
      frame = cv2.flip(frame, 1)
      image_np = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
      # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
      # The model expects a batch of images, so also add an axis with `tf.newaxis`.
      input_tensor = tf.convert_to_tensor(image_np)[tf.newaxis, ...]

      # Pass frame through detector
      detections = detect_fn(input_tensor)

      # Set detection parameters

      score_thresh = 0.05   # Minimum threshold for object detection
      max_detections = 1

      # All outputs are batches tensors.
      # Convert to numpy arrays, and take index [0] to remove the batch dimension.
      # We're only interested in the first num_detections.
      scores = detections['detection_scores'][0, :max_detections].numpy()
      bboxes = detections['detection_boxes'][0, :max_detections].numpy()
      labels = detections['detection_classes'][0, :max_detections].numpy().astype(np.int64)
      labels = [category_index[n]['name'] for n in labels]

      # Display detections
      visualise_on_image(frame, bboxes, labels, scores, score_thresh)

      end_time = time.time()
      fps = int(1/(end_time - start_time))
      start_time = end_time
      cv2.putText(frame, f"FPS: {fps}", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 1)
      #cv2_imshow(frame)
      
      #Write output video
      result.write(frame)

    video_capture.release()


ImportError: ignored

In [ ]:
!pip install utils